<a href="https://colab.research.google.com/github/ndvp39/CloudComputing-tirgul/blob/main/searchEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive, output
import IPython
from IPython.display import HTML, Javascript
import json
from IPython.display import clear_output

drive.mount('/content/drive')

Mounted at /content/drive


Index

In [ ]:
!pip install requests beautifulsoup4
!pip install firebase
!pip install firebase-admin

In [ ]:
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
from collections import defaultdict
from firebase_admin import credentials
from firebase_admin import firestore
import firebase_admin
import requests
import re
import json
import os

In [ ]:
# Function to extract text from a webpage
def get_page_text(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup.get_text()
    except Exception as e:
        return ""

# Function to extract links from a webpage
def get_links(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = [link.get('href') for link in soup.find_all('a', href=True)]
        return links
    except Exception as e:
        print("Error:", e)
        return []

def remove_stop_words(text):
  stop_words = {'a','as','ha','thi','an', 'the', 'and', 'is', 'are', 'or', 'in', 'on', 'at','skip','-','them','they','than','she',',','.','&','for','what'}
  words = [word for word in text if word.lower() not in stop_words]
  return words

def trim_words(words):
    cleaned_words = [word.rstrip(",.\\/") for word in words]
    return cleaned_words

def apply_stemming(words):
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in words]
    return stemmed_words

# Function to create index database
def create_index(url):
    index = defaultdict(list)
    visited = set()
    queue = [url]
    loop = 0
    while queue and loop < 40:
        loop+=1
        current_url = queue.pop(0)
        if current_url in visited:
            continue

        visited.add(current_url)
        text = get_page_text(current_url)
        words = remove_stop_words(text.split())
        words = trim_words(words)
        words = apply_stemming(words)

        for word in words:
            index[word].append(current_url)

        links = get_links(current_url)
        for link in links:
            if link.startswith(url) and link not in visited:
                queue.append(link)
    return index

def update_dict(word, link):
    global link_ids
    if link not in link_ids:
        link_ids[link] = len(link_ids) + 1
    link_id = link_ids[link]
    if link not in index[word]:
        index[word][link] = {'id': link_id, 'counter': 0}
    index[word][link]['counter'] += 1

def create_ranked_words(index):
    sorted_dict = {}
    counter_for_each_word = 0
    for word, links in index.items():
        for link in links.keys():
            counter_for_each_word += index[word][link]['counter']
        sorted_dict[word] = counter_for_each_word
        counter_for_each_word = 0
    sorted_dict = {k: v for k, v in sorted(sorted_dict.items(), key=lambda item: item[1], reverse=True)}
    ranked_dict = {}
    rank = 1
    for word, counter in sorted_dict.items():
        ranked_dict[word] = {'rank': rank, 'counter': counter}
        rank += 1
    return ranked_dict

def create_data_for_db(final_index):
    data_list = []
    for word, doc_ids in final_index.items():
        term_data = {
            'term': word,
            'DocId': doc_ids
        }
        data_list.append(term_data)
    return data_list

def generate_json_file(data_list, is_desktop):
    if is_desktop:
      filepath = os.path.join(os.path.dirname(os.path.abspath(__file__)), "indexDb.json")
    else:
      filepath = "/content/drive/My Drive/CloudComputing/project/indexDb.json"
    with open(filepath, "w") as json_file:
        json.dump(data_list, json_file, indent = 2)

def print_collection_from_db(db, collection_name):
    collection_ref = db.collection(collection_name)
    docs = collection_ref.get()
    i = 0
    for doc in docs:
        term = doc.to_dict()['term']
        print(f"{i} -- Document ID: {doc.id}, Data: {term}")
        i += 1

def upload_to_db(db, collection_name):
  try:
    for data in data_list:
        doc_ref = db.collection(collection_name).add(data)
    print("All documents uploaded successfully.")
  except Exception as e:
      print("Error:", e)


Index - main

In [ ]:
website_url = 'https://www.redhat.com/en'
db_collection_name = "PantherIndex"

link_ids = {}
index_db = create_index(website_url)
index = defaultdict(dict)

for word, links in index_db.items():
    for link in links:
        update_dict(word, link)

ranked_words = create_ranked_words(index)
chosen_words = list(ranked_words.keys())[:110]
final_index = {word:index[word] for word in chosen_words}
data_list = create_data_for_db(final_index)

cred = credentials.Certificate("/content/drive/My Drive/CloudComputing/project/sak.json")
if not firebase_admin._apps:
  firebase_admin.initialize_app(cred)
db = firestore.client()

# GET INDEX FROM DB
# index = det_index_from_db()

print_collection_from_db(db, db_collection_name)

0 -- Document ID: 0T4jk18lvS0dmor7kkEo, Data: triallearn
1 -- Document ID: 0dC3yFw12YLEqBoyIxaC, Data: buy
2 -- Document ID: 0rOjcv363Sdx7bxHCmXz, Data: from
3 -- Document ID: 14q8ET1SDE7sGorNMlJH, Data: organ
4 -- Document ID: 1OMqSz8Oy3XUhnh2eDe7, Data: sellr
5 -- Document ID: 239jIl6zY4B8DRyc8od2, Data: secur
6 -- Document ID: 2G9rtYGHlieDnuJOflVa, Data: across
7 -- Document ID: 2XtPvNMhEf42gc6jqrfb, Data: kubernet
8 -- Document ID: 37PzlFuvqdnSJ8DaMlQa, Data: how
9 -- Document ID: 3JlA8NiuJus3xFV9nlux, Data: technolog
10 -- Document ID: 3PUtUH4fOrAfqNvfzPjm, Data: build
11 -- Document ID: 49ptfniPoRUQURAU8Kug, Data: i
12 -- Document ID: 4kd7oscDguWFo1XAx1FM, Data: make
13 -- Document ID: 5iIeVk2PXs2CbEfJl1sH, Data: managementr
14 -- Document ID: 5yndNLJlUQcJnN2hIut3, Data: servic
15 -- Document ID: 5zueNX9sfVLPFGNcm95N, Data: hubinteract
16 -- Document ID: 609wbzfpwg7SMBUa7ZLy, Data: by
17 -- Document ID: 60gLRy4xWThsl7gIfyK2, Data: contain
18 -- Document ID: 695WTouVyvzM1DjRUnns, 

ChatBot

In [ ]:
def getLinkFromString(w,linkID):
  for word, links in final_index.items(): # should be from database directly and not from database
    if(word == w):
      return list(links.keys())[linkID]

def get_first_paragraph_from_link(url):
    try:
        # Make an HTTP request
        response = requests.get(url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find the first <p> tag with the specified ID
            h2_tag = soup.find('h2', {'id': 'overview'})
            if h2_tag:

              para = h2_tag.find_next('p')
              if para:
                return para.get_text()
              else:
                return "Paragraph after h2 with id: overview not found."

            else:

              div_tags = soup.find_all('div', {'class': 'rh-generic--component'})
              # Check if there is at least a second occurrence
              if len(div_tags) >= 2:
                second_div_tag = div_tags[1]  # Select the second occurrence
                para = second_div_tag.find_next('p').get_text()
              elif len(div_tags) == 1:
                first_div_tag = div_tags[0]  # Select the first occurrence
                para = first_div_tag.find_next('p').get_text()
              else:
                return "" # maybe return something more generall @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@?????????????????????????????????????????

            return para



        else:
            return f"Failed to fetch the page. Status code: {response.status_code}"

    except Exception as e:
        return f"An error occurred: {e}"





!pip install nltk

import nltk
from nltk.chat.util import Chat, reflections

cloud_explanation = """Private Cloud: A private cloud is a cloud computing environment that is dedicated solely to one organization.
It is usually managed either by the organization itself or by a third-party service provider. The infrastructure and services are not shared
with other organizations, offering greater control, security, and customization options. Private clouds are typically used by large enterprises
or organizations with specific compliance or security requirements.
Public Cloud: A public cloud is a cloud computing environment that is hosted and managed by a third-party service provider and made available
to multiple organizations or individuals over the internet. The infrastructure and resources are shared among multiple users,
 offering scalability, flexibility, and cost-effectiveness. Public clouds are accessible to anyone who wants to use them and are often used
 for general-purpose computing needs such as web hosting, development, and testing.
Hybrid Cloud: A hybrid cloud is a computing environment that combines elements of both private and public clouds. It allows organizations
 to leverage the benefits of both environments by integrating on-premises infrastructure with public cloud resources. This integration enables
 data and applications to be dynamically moved between the two environments based on changing workload demands, security requirements, and cost
 considerations. Hybrid clouds offer greater flexibility, scalability, and customization options compared to using either private or public clouds alone.
 They are commonly used by organizations that want to retain control over sensitive data while taking advantage of the scalability and cost-effectiveness of
 public cloud services."""


# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

# Define some patterns and responses. ______________ take this patterns fron database ______________
patterns = [
    (r'hi|hello|hey', ['Hello!', 'Hi there!', 'Hey!']),
    (r'how are you?', ['I\'m good, thank you!', 'I\'m doing well, thanks for asking.']),
    (r'what is your name?', ['You can call me ChatGPT.', 'I go by the name ChatGPT.']),
    (r'your name (.*)', ['You can call me ChatGPT.', 'I go by the name ChatGPT.']),
    (r'give me good clud website', ['Sure, here: www.redhat.com', 'ofCourse: you can go to amazon, which works with RedHat website check both']),
    (r'What kinds of clouds are there ?', ['There are three main kinds of clouds: private, public, and hybrid would you like me to explain further? ', 'public, hybrid, private - in secured order']),
    (r'(.*)kinds of clouds(.*)?(.*)', ['There are three main kinds of clouds: private, public, and hybrid would you like me to explain further? ', 'public, hybrid, private - in secured order']),
    (r'please explain further about clouds(.*)', [cloud_explanation, "for the moment I too tired, may be later I will explain. You should've listen in class "])
    # You can add more patterns and responses here
]

for word, links in final_index.items(): # take the index from DB
  answers = []
  for i in range(len(links)):
    link = getLinkFromString(word,i)
    first_para = get_first_paragraph_from_link(link)
    if first_para == "": # also need to remove dates
      continue
    answers.append(first_para + f"you can read more in the link: {link}")
    print(first_para)
  to_append = ( r'(.*)' + word + r'(.*)', answers)
  patterns.append(to_append)


# Create a chatbot
chatbot = Chat(patterns, reflections)

# Start the conversation
#print("Hello! I'm ChatGPT. How can I help you today?")
#while True:
    #user_input = input("You: ")
    #response = chatbot.respond(user_input)
    #print("ChatGPT:", response)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Red Hat® Summit and AnsibleFest bring together IT professionals, customers, partners, and peers to provide you the tools, connections, and knowledge to support your technology goals. Register now and join us May 6-9 in Denver, Colorado, to solve difficult challenges, grow your expertise, and learn ways to meet the demands of your customers with less, not more, complexity. You’ll leave with new ideas to help you and your company excel.
The world's leading enterprise Linux platform. Deploy it on physical systems, as a guest on the most widely available hypervisors, or in the cloud.
The Red Hat Customer Portal provides existing customers quick access to troubleshooting resources, knowledge base articles, product documentation, and technical support.
We’re here to help you address change with open principles so that you can navigate today’s need for transformation, and prepare for the future.
Red Hat® Summit and AnsibleFest bring together IT professionals, customers, partners, and peers to

SearchEngine

In [ ]:

def switchToScreen(css_filename_, html_filename_, js_filename_):

  clear_output(wait=True)

  if(css_filename_ is None and js_filename_ is None):
      with open(html_filename_, 'r') as file:
        html_content_ = file.read();
        display(IPython.display.HTML(html_content_))
  else:
    with open(css_filename_, 'r') as file:
      css_content_ = file.read();
      display(IPython.display.HTML(css_content_))
    with open(html_filename_, 'r') as file:
      html_content_ = file.read();
      display(IPython.display.HTML(html_content_))
    with open(js_filename_, 'r') as file:
      js_content_ = file.read();
      display(IPython.display.Javascript(js_content_))

In [ ]:
def get_title_from_link(link):


  try:
        # Make an HTTP request
        response = requests.get(link)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the HTML content
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find the <h1> tag
            h1_tag = soup.find('h1')
            if h1_tag:
              return h1_tag.get_text()
            else:
              h2_tag = soup.find('h2')
              if h2_tag:
                return h2_tag.get_text()
        else:
            return f"Failed to fetch the page. Status code: {response.status_code}"

  except Exception as e:
        return f"An error occurred: {e}"




def get_titles_links_of_search(search_string):
  titles = []
  links1 = []
  for word, links in final_index.items():
    if word in search_string or word==search_string:
      for i in range(min(10, len(links))): # turn it to pages and remove min @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
        curr_link = getLinkFromString(word, i)
        titles.append(get_title_from_link(curr_link))
        links1.append(curr_link)
  return titles, links1


In [ ]:
def receive_search_string(search_string):
  original_search_string = search_string
  search_string = search_string.lower()
  print(1231)
  css_filename = "/content/drive/My Drive/CloudComputing/project/style.css"
  html_filename = "/content/drive/My Drive/CloudComputing/project/results.html"
  js_filename = "/content/drive/My Drive/CloudComputing/project/script.js"

  switchToScreen(css_filename, html_filename, js_filename)

  js_code = f"""
    <script>
      document.getElementById("searchbartext").value = "{original_search_string}";
    </script>
    """

  titles, links = get_titles_links_of_search(search_string)

  # Convert Python arrays to JavaScript arrays
  titles_js_array = ', '.join(f'"{title}"' for title in titles)
  links_js_array = ', '.join(f'"{link}"' for link in links)

  js_code_2 = f"""
    <script>
      renderResults_title_link([ {titles_js_array} ], [ {links_js_array} ]);
    </script>
  """

  display(HTML(js_code))
  display(HTML(js_code_2))

# Register the function to be called from JavaScript
output.register_callback('notebook.receive_search_string', receive_search_string)

def open_chatbot_page():
  css_filename = "/content/drive/My Drive/CloudComputing/project/style.css"
  html_filename = "/content/drive/My Drive/CloudComputing/project/chatbot.html"
  js_filename = "/content/drive/My Drive/CloudComputing/project/script.js"
  switchToScreen(css_filename, html_filename, js_filename)

# Register the function to be called from JavaScript
output.register_callback('notebook.open_chatbot_page', open_chatbot_page)

def receive_question_chatbot(question):
  response = chatbot.respond(question)
  if response is None:
    response = "Sorry, i dont have response for your answer :/"
  js_code = f"""
      <script>
        ChatbotResponse("{response}");
      </script>
    """
  #print(response)
  display(HTML(js_code))

# Register the function to be called from JavaScript
output.register_callback('notebook.receive_question_chatbot', receive_question_chatbot)

def back_to_home_page():
  css_filename = "/content/drive/My Drive/CloudComputing/project/style.css"
  html_filename = "/content/drive/My Drive/CloudComputing/project/index.html"
  js_filename = "/content/drive/My Drive/CloudComputing/project/script.js"
  switchToScreen(css_filename, html_filename, js_filename)

# Register the function to be called from JavaScript
output.register_callback('notebook.back_to_home_page', back_to_home_page)

css_filename = "/content/drive/My Drive/CloudComputing/project/style.css"
html_filename = "/content/drive/My Drive/CloudComputing/project/index.html"
js_filename = "/content/drive/My Drive/CloudComputing/project/script.js"

switchToScreen(css_filename, html_filename, js_filename)


<IPython.core.display.Javascript object>

In [ ]:
def open_statistics_screen():
  html_filename = "/content/drive/My Drive/CloudComputing/project/statistics.html"
  switchToScreen(None, html_filename, None)

# Register the function to be called from JavaScript
output.register_callback('notebook.open_statistics_screen', open_statistics_screen)

def back_to_admin():
  html_filename = "/content/drive/My Drive/CloudComputing/project/admin.html"
  switchToScreen(None, html_filename, None)

# Register the function to be called from JavaScript
output.register_callback('notebook.back_to_admin', back_to_admin)

html_filename = "/content/drive/My Drive/CloudComputing/project/admin.html"
switchToScreen(None, html_filename, None)